In [17]:
import random
from parsl import DataFlowKernel, ThreadPoolExecutor, App
import pprint
import json

workers = ThreadPoolExecutor(max_workers=8)
dfk = DataFlowKernel(workers)

In [18]:
@App('python', dfk)
def add(x, y):  
    import time
    time.sleep(1)
    return x+y

In [19]:
fibs = [0,1]
for i in range(0,99):
    cur = add(fibs[i], fibs[i+1])
    #print("Index[{0}] : {1}".format(i+2, cur))
    fibs.extend([cur])



In [20]:
from parsl.dataflow.states import States
from IPython.display import Javascript
import json

class DFKListener():
    def __init__(self, dfk):
        self.dfk = dfk
        self.nodes_list = []
        self.edges_list = []
        self.color_lookup_table = {
                     States.unsched : "Pink",
                      States.pending : "Yellow",
                      States.runnable: "Blue",
                      States.running : "Lime",
                      States.done: "Green",
                      States.failed  : "Red",
                      States.dep_fail: "orange"
        }
        self.state_lookup_table = {
                     States.unsched : "Unscheduled",
                      States.pending : "Pending",
                      States.runnable: "Runnable",
                      States.running : "Running",
                      States.done: "Finished",
                      States.failed  : "Failed",
                      States.dep_fail: "Dependency Failure"
        }


    @property
    def nodes(self):
        return self.nodes_list
        
    @property
    def edges(self):
        return self.edges_list
    
    
    def create_nodes(self, dfke=None):
        dfk = dfke if dfke else self.dfk
        for task in dfk.tasks:
            state = dfk.tasks[task]['status']
            tid = dfk.tasks[task]['app_fu'].tid
            task = {"id": tid, "label": tid, "color": self.color_lookup_table[state], "font": {"background": "white"}, "title": self.state_lookup_table[state]}
            self.nodes_list.append(task)
        return self.nodes
    
    def create_edges(self, dfke=None):
        dfk = dfke if dfke else self.dfk
        for task in dfk.tasks:
            deps = state = dfk.tasks[task]['depends']
            for dep in deps:
                edge = {"from" : dep.tid, "to": task, "arrows":'to'}
                self.edges_list.append(edge)
        return self.edges
    
    def update(self):
        self.create_nodes()
        self.create_edges()
        return json.dumps({"nodes":self.nodes,"edges":self.edges})
    
    def set_javascript(self):
        self.update()
        nodes = json.dumps(self.nodes)
        edges = json.dumps(self.edges)
        Javascript("""window.nodesList={}; window.edgesList={};""".format(nodes,edges))

In [29]:
dfkl = DFKListener(dfk)

In [30]:
dfkl.set_javascript()

In [31]:
dfkl.update()


'{"nodes": [{"id": 0, "label": 0, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 1, "label": 1, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 2, "label": 2, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 3, "label": 3, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 4, "label": 4, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 5, "label": 5, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 6, "label": 6, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 7, "label": 7, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 8, "label": 8, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 9, "label": 9, "color": "Green", "font": {"background": "white"}, "title": "Finished"}, {"id": 10, "label": 10, "color": "Gree

In [35]:
%%javascript
require.config({
  paths: {
    vis: 'https://cdnjs.cloudflare.com/ajax/libs/vis/4.18.1/vis.min'
  }
});
require(['vis'], function (vis) {
    element.append('<div id="mynetwork" style="width:950px;height:750px;border:1px solid lightgray;"></div>')
    var nodes = new vis.DataSet(window.nodesList);
    var edges = new vis.DataSet(window.edgesList);
    var container = document.getElementById('mynetwork');
    var data = {
         nodes: nodes,
         edges: edges
     };
     var options = {interaction:{hover:true}, 
                   layout: {
                    hierarchical: {
                        direction: 'UD'
                    }}
                   };
     var network = new vis.Network(container, data, options);
     network.on("hoverNode", function (params) {
             
     });
     network.on("hoverEdge", function (params) {
            
     });
     network.on("blurNode", function (params) {
             
     });
     network.on("blurEdge", function (params) {
             
     });
});


<IPython.core.display.Javascript object>

In [28]:
%%javascript 
$(".vis-tooltip").insertBefore($(".vis-network"))

<IPython.core.display.Javascript object>